In [2]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
import numpy as np
from operator import itemgetter

import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy.sparse as sps

In [3]:
from Data_Handler.DataReader import DataReader 
dataReader=DataReader()
URM_all=dataReader.load_augmented_binary_urm()
powerful_urm=dataReader.load_powerful_binary_urm()

#URM_train, URM_validation = split_train_in_two_percentage_global_sample(urm, train_percentage = 0.9)
#train_powerful, val_powerful = split_train_in_two_percentage_global_sample(powerful_urm, train_percentage = 0.9)

In [4]:
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 319 ( 0.8%) Users that have less than 1 test interactions


In [10]:
MAP=[]

for topK in [10, 50, 100, 200, 500]:
    for shrink in [0, 10, 50, 100, 200, 500]:
        recommender = ItemKNNCFRecommender(URM_train)
        recommender.fit(topK=topK, shrink=shrink)

        result_df, _ = evaluator_validation.evaluateRecommender(recommender)
        print(result_df)
        #result_df = result_df.drop(['cutoff'], axis = 1)
        result_df = result_df.to_dict()
        #print(result_df)
        #print(type(result_df))
        #MAP.append(('topK={}'.format(topK),'shrink={}'.format(shrink),result_df["MAP"]))
        #print(MAP)

Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 24507 (100.0%), 1322.78 column/sec. Elapsed time 18.53 sec
EvaluatorHoldout: Processed 41310 (100.0%) in 9.66 sec. Users per second: 4276
       PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
cutoff                                                                      
10      0.058681                 0.079174  0.068587  0.029221    0.038956   

             MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
cutoff                                                        ...   
10      0.183422  0.082124  0.063248  0.366618      0.229251  ...   

       COVERAGE_USER COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI  \
cutoff                                                              
10          0.992337          0.363809    0.992337       0.043861   

       SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI  \
cutoff                                                      

KeyboardInterrupt: 

In [34]:
MAP

['topK=10',
 'shrink=0',
 0.029331034046662496,
 'topK=10',
 'shrink=10',
 0.029323209591231945,
 'topK=10',
 'shrink=50',
 0.029311150973356393,
 'topK=10',
 'shrink=100',
 0.029297173240092123,
 'topK=10',
 'shrink=200',
 0.029306732299066364,
 'topK=10',
 'shrink=500',
 0.029275828487013125,
 'topK=50',
 'shrink=0',
 0.0309715077193209,
 'topK=50',
 'shrink=10',
 0.03097090228982921,
 'topK=50',
 'shrink=50',
 0.030967871298373986,
 'topK=50',
 'shrink=100',
 0.03096935508112821,
 'topK=50',
 'shrink=200',
 0.03094246344070651,
 'topK=50',
 'shrink=500',
 0.030922445827513335,
 'topK=100',
 'shrink=0',
 0.031228393374639618,
 'topK=100',
 'shrink=10',
 0.031221891254098806,
 'topK=100',
 'shrink=50',
 0.031210596630581736,
 'topK=100',
 'shrink=100',
 0.031197507821571144,
 'topK=100',
 'shrink=200',
 0.031215748547256134,
 'topK=100',
 'shrink=500',
 0.031150191104298265,
 'topK=200',
 'shrink=0',
 0.03089011685365652,
 'topK=200',
 'shrink=10',
 0.030884634353259624,
 'topK=200',


In [28]:
max_tuple = max(MAP, key=itemgetter(2))
print('The best combination for the ICF recommender is ({},{}), with a MAP = {}'.format(max_tuple[0],max_tuple[1], max_tuple[2]))

TypeError: 'str' object is not callable

In [6]:
MAP=[]

for num_factors in np.logspace(1,15,10, base=2).astype(int):
    recommender = PureSVDRecommender(URM_train)
    recommender.fit(num_factors = num_factors)

    result_dict, _ = evaluator_validation.evaluateRecommender(recommender)

    MAP.append(('number of factors={}'.format(num_factors),result_dict[10]["MAP"]))

PureSVDRecommender: Computing SVD decomposition...
PureSVDRecommender: Computing SVD decomposition... done in 0.27 sec
EvaluatorHoldout: Processed 41311 (100.0%) in 22.34 sec. Users per second: 1849


KeyError: 10